<a href="https://colab.research.google.com/github/kh-w/Solving-a-n2-x-n2-Sudoku-using-the-Grovers-Algorithm/blob/main/Grovers_search_Sudoku_Check_ver_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit-aer pylatexenc

from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister, transpile
from qiskit.circuit.library import MCXGate
from qiskit.quantum_info import Statevector, Operator
from qiskit_aer import AerSimulator, Aer
from qiskit.visualization import plot_histogram

import numpy as np
import matplotlib.pyplot as plt
from collections import Counter


In [ ]:
# Target initial Sudoku (Top 3 rows only)
# i_sudoku = ['001110xxxx1011001101xx10'] # correspond to row1:143x row2:x341 row3:42x3 row4:3x24
i_sudoku = ['001110xxxx1011001101xx10a'] # correspond to row1:143x row2:x341 row3:42x3 row4:3x24
ctrl_qubits_0 = [i for i, c in enumerate(i_sudoku[0]) if c == '0'] # These are zeros in i_sudoku
ctrl_qubits_1 = [i for i, c in enumerate(i_sudoku[0]) if c == '1'] # These are ones in i_sudoku

n_qubits = len(i_sudoku[0]) - 1
n_ancilla = i_sudoku[0].count('a')

def grover_oracle():

    oracle = QuantumCircuit(n_qubits + n_ancilla)

    oracle.barrier()

    for i in ctrl_qubits_0:
        oracle.x(i)

    oracle.mcx(ctrl_qubits_0 + ctrl_qubits_1,
               n_qubits - 1 + n_ancilla)

    for i in ctrl_qubits_0:
        oracle.x(i)

    return oracle

def diffuser():
    qc = QuantumCircuit(n_qubits)
    qc.barrier()
    qc.barrier()
    qc.h(range(n_qubits))
    qc.x(range(n_qubits))
    qc.h(n_qubits - 1)
    qc.mcx(list(range(n_qubits - 1)), n_qubits - 1)
    qc.h(n_qubits - 1)
    qc.x(range(n_qubits))
    qc.h(range(n_qubits))
    qc.barrier()
    return qc

grover = QuantumCircuit(n_qubits + n_ancilla, n_qubits)
grover.h(range(n_qubits))

N = 2**n_qubits
M = len(i_sudoku) * (2**(i_sudoku[0].count('x')))
iterations = int(np.floor((np.pi / 4) * np.sqrt(N / M)))
print(f"Grover iterations: {iterations}")

for _ in range(iterations):
    grover = grover.compose(grover_oracle())
    grover = grover.compose(diffuser())

grover.measure(range(n_qubits), range(n_qubits))

In [ ]:
M

In [ ]:
import time
start_time = time.time()

backend = Aer.get_backend('aer_simulator')
grover_t = transpile(grover, backend)
result = backend.run(grover_t, shots=20000).result()
counts = result.get_counts()

end_time = time.time()
print(f"Elapsed time: {(end_time - start_time)/60:.4f} minutes")

In [ ]:
sum(counts.values())

In [ ]:
most_common = Counter(counts).most_common()
top_n = 500
print(i_sudoku[0][:-1][::-1])
for outcome, count in most_common[:top_n]:
    probability = count / sum(counts.values())
    print(f"Outcome: {outcome}, Counts: {count}, Probability: {probability:.4f}")

In [ ]:
break

In [ ]:
targets =  ['00011011',
            '00011110',
            '00100111',
            '00101101',
            '00110110',
            '00111001',
            '01001011',
            '01001110',
            '01100011',
            '01101100',
            '01110010',
            '01111000',
            '10000111',
            '10001101',
            '10010011',
            '10011100',
            '10110001',
            '10110100',
            '11000110',
            '11001001',
            '11010010',
            '11011000',
            '11100001',
            '11100100'] # 1234, 1243, ... (24 of such) encoded in binary (left to right)

In [ ]:
n_qubits_rowcheck = 8

def grover_oracle_rowcheck():
    oracle = QuantumCircuit(n_qubits_rowcheck)

    for target in targets:
        qc = QuantumCircuit(n_qubits_rowcheck)
        qc.barrier()
        ctrl_qubits = []
        for i, bit in enumerate(reversed(target)):
            if bit == '0':
                qc.x(i)
            ctrl_qubits.append(i)

        qc.h(n_qubits_rowcheck - 1)
        qc.mcx(ctrl_qubits[:-1], ctrl_qubits[-1])
        qc.h(n_qubits_rowcheck - 1)

        for i, bit in enumerate(reversed(target)):
            if bit == '0':
                qc.x(i)

        oracle = oracle.compose(qc)

    return oracle

def diffuser_rowcheck():
    qc = QuantumCircuit(n_qubits_rowcheck)
    qc.barrier()
    qc.barrier()
    qc.h(range(n_qubits_rowcheck))
    qc.x(range(n_qubits_rowcheck))
    qc.h(n_qubits_rowcheck - 1)
    qc.mcx(list(range(n_qubits_rowcheck - 1)), n_qubits_rowcheck - 1)
    qc.h(n_qubits_rowcheck - 1)
    qc.x(range(n_qubits_rowcheck))
    qc.h(range(n_qubits_rowcheck))
    qc.barrier()
    return qc


grover_rowcheck = QuantumCircuit(n_qubits_rowcheck, n_qubits_rowcheck)
grover_rowcheck.h(range(n_qubits_rowcheck))

N = 2**n_qubits_rowcheck
M = len(targets)
iterations = int(np.floor((np.pi / 4) * np.sqrt(N / M)))
print(f"Grover iterations: {iterations}")

oracle_rowcheck = grover_oracle_rowcheck()
diff_rowcheck = diffuser_rowcheck()

for _ in range(iterations):
    grover_rowcheck = grover_rowcheck.compose(oracle_rowcheck)
    grover_rowcheck = grover_rowcheck.compose(diff_rowcheck)

# sv = Statevector(grover)
grover_rowcheck.measure(range(n_qubits_rowcheck), range(n_qubits_rowcheck))

grover_rowcheck.draw(output="mpl", style="bw", scale=0.5, fold=-1)

In [ ]:
import time
start_time = time.time()

backend = Aer.get_backend('aer_simulator')
grover_t = transpile(grover_rowcheck, backend)
result = backend.run(grover_t, shots=20000).result()
counts = result.get_counts()

end_time = time.time()
print(f"Elapsed time: {(end_time - start_time)/60:.4f} minutes")

In [ ]:
sum(counts.values())

In [ ]:
most_common = Counter(counts).most_common()
top_n = 500
for outcome, count in most_common[:top_n]:
    probability = count / sum(counts.values())
    print(f"Outcome: {outcome}, Counts: {count}, Probability: {probability:.4f}")